In [58]:
from itertools import combinations, permutations

In [101]:
class Calculator:
    
    def __init__(self, opcoder):
        self.opcoder = opcoder

    def instruction_map(self):
        return {
            1: {"fn": self.add, "number_of_parameters": 3},
            2: {"fn": self.multiply, "number_of_parameters": 3},
            3: {"fn": self.store, "number_of_parameters": 1},
            4: {"fn": self.output, "number_of_parameters": 1},
            5: {"fn": self.jump_if_true, "number_of_parameters": 2},
            6: {"fn": self.jump_if_false, "number_of_parameters": 2},
            7: {"fn": self.less_than, "number_of_parameters": 3},
            8: {"fn": self.equals, "number_of_parameters": 3},
            99: {"fn" :self. exit, "number_of_parameters": 0},
        }
        
    def get_value_for_mode(self, param, mode):
        return self.opcoder.program[param] if mode == 0 else param
    
    def add(self, param_mode1, param_mode2, output_mode):
        result = self.get_value_for_mode(*param_mode1) + self.get_value_for_mode(*param_mode2) 
        output_position = output_mode[0]
        self.opcoder.program[output_position] = result
        self.opcoder.pointer += 3 + 1
        
    def multiply(self, param_mode1, param_mode2, output_mode):
        result = self.get_value_for_mode(*param_mode1) * self.get_value_for_mode(*param_mode2) 
        output_position = output_mode[0]
        self.opcoder.program[output_position] = result
        self.opcoder.pointer += 3 + 1
    
    def store(self, position_mode):
        output_position = position_mode[0]
        self.opcoder.program[output_position] = self.opcoder.input.pop(0)
        self.opcoder.pointer += 1 + 1
        
    def output(self, position_mode):
        self.opcoder.output.append(self.get_value_for_mode(*position_mode))
        self.opcoder.pointer += 1 + 1
    
    def jump_if_true(self, test_mode, jump_mode):
        if self.get_value_for_mode(*test_mode) != 0:
            self.opcoder.pointer = self.get_value_for_mode(*jump_mode)
        else:
            self.opcoder.pointer += 2 + 1
    
    def jump_if_false(self, test_mode, jump_mode):
        if self.get_value_for_mode(*test_mode) == 0:
            self.opcoder.pointer = self.get_value_for_mode(*jump_mode)
        else:
            self.opcoder.pointer += 2 + 1

    def less_than(self, test1_mode, test2_mode, output_mode):
        if self.get_value_for_mode(*test1_mode) < self.get_value_for_mode(*test2_mode):
            self.opcoder.program[output_mode[0]] = 1
        else:
            self.opcoder.program[output_mode[0]] = 0
        self.opcoder.pointer += 3 + 1

    def equals(self, test1_mode, test2_mode, output_mode):
        if self.get_value_for_mode(*test1_mode) == self.get_value_for_mode(*test2_mode):
            self.opcoder.program[output_mode[0]] = 1
        else:
            self.opcoder.program[output_mode[0]] = 0
        self.opcoder.pointer += 3 + 1
    
    def exit(self):
        self.opcoder.pointer += 1

In [102]:
class Opcoder:
    
    def __init__(self, program):
        self.program = list(program)
        self.done = False
        self.pointer = 0
        self.input = []
        self.output = []
    
    def read_opcode(self, opcode: str):
        instruction = int(opcode[-2:])
        parameter_mode_str = opcode[:-2]
        parameter_modes = []
        for i in range(1, 4):
            try:
                parameter_modes.append(int(parameter_mode_str[-i]))
            except IndexError:
                parameter_modes.append(0)
        return instruction, parameter_modes

    def add_input(self, value):
        self.input.append(value)
        self.run_program()
        
    def run_program(self):
        while True:
            # find opcode at pointer
            opcode = str(self.program[self.pointer]).zfill(5)
            if opcode == "00099":
                self.done = True
                return
            if opcode == "00003" and len(self.input) == 0:
                return
            
            instruction, parameter_modes = self.read_opcode(opcode)
    
            calculator = Calculator(self)
            calc_config = calculator.instruction_map()[instruction]
            
            first_param_pointer = self.pointer + 1
            parameters = self.program[first_param_pointer: first_param_pointer + calc_config["number_of_parameters"]]
            parameters_and_modes = list(zip(parameters, parameter_modes))
            
            calc_config["fn"](*parameters_and_modes)